In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
!pwd

/Users/malte.luecken/helmholtz_munich/benchmarking_data_integration/Benchmarking_data_integration/notebooks/analysis


In [8]:
files = !ls ../../../Paper/Supplementary\ Files/Results/*.csv
#files = !ls ../../R/visualization/*.csv

In [9]:
data = {file:pd.read_csv(file) for file in files}

In [10]:
data[files[0]]

,Unnamed: 0,Method,Output,Features,Scaling,Overall Score,Batch Correction,PCR batch,Batch ASW,graph iLISI,...,Bio conservation,NMI cluster/label,ARI cluster/label,Cell type ASW,isolated label F1,isolated label silhouette,graph cLISI,CC conservation,HVG conservation,trajectory conservation
0,53,Scanorama,embed,HVG,scaled,0.684833,0.585777,0.885319,0.952180,0.309590,...,0.750871,0.627134,0.416023,0.503741,0.167785,0.586023,0.826255,0.719294,NaN,0.545590
1,54,Scanorama,embed,FULL,scaled,0.676200,0.547736,0.835144,0.940894,0.227042,...,0.761843,0.638806,0.381296,0.489245,0.148867,0.474113,0.933437,0.661380,NaN,0.842511
2,193,scVI,embed,HVG,unscaled,0.674271,0.563097,0.843787,0.919227,0.252864,...,0.748387,0.637718,0.394966,0.515436,0.211710,0.397632,0.927769,0.551211,NaN,0.734517
3,194,scVI,embed,FULL,unscaled,0.667738,0.473860,0.704054,0.895534,0.198181,...,0.796991,0.633000,0.351380,0.514931,0.232658,0.414654,0.959683,0.591230,NaN,0.937461
4,165,BBKNN,graph,HVG,unscaled,0.662366,0.871410,NaN,NaN,1.000000,...,0.523004,0.555303,0.468053,NaN,0.078734,NaN,0.000000,NaN,NaN,0.684339
5,236,ComBat,gene,FULL,unscaled,0.639724,0.520101,0.999995,0.913132,0.143482,...,0.719472,0.614375,0.346681,0.470106,0.211682,0.383198,0.954763,0.867913,0.239344,0.908189
6,25,MNN,gene,HVG,scaled,0.622739,0.566140,0.905940,0.932919,0.237516,...,0.660472,0.631690,0.378881,0.496933,0.127805,0.402252,0.932058,0.762060,0.290062,0.636212
7,11,MNN,gene,HVG,unscaled,0.621272,0.391765,0.120337,0.923232,0.165583,...,0.774277,0.640366,0.354824,0.526847,0.209784,0.485932,0.957292,0.766928,0.343757,0.869389
8,81,Scanorama,gene,HVG,scaled,0.618259,0.487988,0.558651,0.876135,0.270893,...,0.705106,0.619437,0.387634,0.503736,0.257360,0.586023,0.892119,0.738449,0.291859,0.429069
9,249,ComBat,gene,HVG,scaled,0.617717,0.544625,1.000000,0.923294,0.204919,...,0.666445,0.607445,0.347841,0.470197,0.121891,0.381516,0.926552,0.769218,0.253202,0.899172


# Unscaled vs scaled

In [11]:
batch_corr = []
bio_cons = []
overall = []

for file in data:
    data[file] = data[file].loc[~np.isnan(data[file]['Overall Score']),:]
    
    data[file]['method_id'] = ['_'.join(data[file][['Method', 'Output', 'Features']].values[i]) for i in range(data[file].shape[0])]
    
    for meth in data[file]['method_id'].unique():
        tmpDat = data[file][['Scaling', 'Batch Correction', 'Bio conservation', 'Overall Score']].loc[data[file]['method_id'].isin([meth])]
        if tmpDat.shape[0] == 2:
            overall.append(tmpDat['Overall Score'].loc[tmpDat['Scaling'] == 'unscaled'].values[0] > tmpDat['Overall Score'].loc[tmpDat['Scaling'] == 'scaled'].values[0])
            bio_cons.append(tmpDat['Bio conservation'].loc[tmpDat['Scaling'] == 'unscaled'].values[0] > tmpDat['Bio conservation'].loc[tmpDat['Scaling'] == 'scaled'].values[0])
            batch_corr.append(tmpDat['Batch Correction'].loc[tmpDat['Scaling'] == 'unscaled'].values[0] > tmpDat['Batch Correction'].loc[tmpDat['Scaling'] == 'scaled'].values[0])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
np.mean(overall)
np.mean(bio_cons)
np.mean(batch_corr)

0.5151515151515151

0.7171717171717171

0.37373737373737376

Unscaled scores tend to have a higher bio conservation score, but a lower batch removal score (72% vs 37% of cases).

In [13]:
len(overall)

99

# HVG vs Full feature

In [14]:
batch_corr = []
bio_cons = []
overall = []

for file in data:
    data[file] = data[file].loc[~np.isnan(data[file]['Overall Score']),:]
    
    data[file]['method_id'] = ['_'.join(data[file][['Method', 'Output', 'Scaling']].values[i]) for i in range(data[file].shape[0])]
    
    for meth in data[file]['method_id'].unique():
        tmpDat = data[file][['Features', 'Batch Correction', 'Bio conservation', 'Overall Score']].loc[data[file]['method_id'].isin([meth])]
        if tmpDat.shape[0] == 2:
            overall.append(tmpDat['Overall Score'].loc[tmpDat['Features'] == 'HVG'].values[0] > tmpDat['Overall Score'].loc[tmpDat['Features'] == 'FULL'].values[0])
            bio_cons.append(tmpDat['Bio conservation'].loc[tmpDat['Features'] == 'HVG'].values[0] > tmpDat['Bio conservation'].loc[tmpDat['Features'] == 'FULL'].values[0])
            batch_corr.append(tmpDat['Batch Correction'].loc[tmpDat['Features'] == 'HVG'].values[0] > tmpDat['Batch Correction'].loc[tmpDat['Features'] == 'FULL'].values[0])

In [15]:
np.mean(overall)
np.mean(bio_cons)
np.mean(batch_corr)
len(overall)

0.717948717948718

0.5982905982905983

0.8034188034188035

117

# Method top performer

In [16]:
topMeth = dict()
topMeth_bio = dict()
topMeth_batch = dict()

for file in data:
    data[file] = data[file].loc[~np.isnan(data[file]['Overall Score']),:]
    
    data[file]['method_id'] = ['_'.join(data[file][['Method', 'Output']].values[i]) for i in range(data[file].shape[0])]
    
    for meth in data[file]['method_id'].unique():
        tmpDat = data[file][['Features', 'Scaling', 'Batch Correction', 'Bio conservation', 'Overall Score']].loc[data[file]['method_id'].isin([meth])]

        if meth not in topMeth:
            topMeth[meth] = {'HVG_scaled':0, 'HVG_unscaled':0, 'FULL_scaled':0, 'FULL_unscaled':0}
            topMeth_bio[meth] = {'HVG_scaled':0, 'HVG_unscaled':0, 'FULL_scaled':0, 'FULL_unscaled':0}
            topMeth_batch[meth] = {'HVG_scaled':0, 'HVG_unscaled':0, 'FULL_scaled':0, 'FULL_unscaled':0}

        tmpDat['preproc'] = ['_'.join(tmpDat[['Features', 'Scaling']].values[i]) for i in range(tmpDat.shape[0])]
        
        topVal = tmpDat.sort_values(by='Overall Score', ascending=False)['preproc'].values[0]
        topMeth[meth][topVal] += 1
        
        topVal = tmpDat.sort_values(by='Bio conservation', ascending=False)['preproc'].values[0]
        topMeth_bio[meth][topVal] += 1
        
        topVal = tmpDat.sort_values(by='Batch Correction', ascending=False)['preproc'].values[0]
        topMeth_batch[meth][topVal] += 1        

In [17]:
topMeth

{'Scanorama_embed': {'HVG_scaled': 6,
  'HVG_unscaled': 1,
  'FULL_scaled': 0,
  'FULL_unscaled': 0},
 'scVI_embed': {'HVG_scaled': 0,
  'HVG_unscaled': 5,
  'FULL_scaled': 0,
  'FULL_unscaled': 2},
 'BBKNN_graph': {'HVG_scaled': 0,
  'HVG_unscaled': 6,
  'FULL_scaled': 0,
  'FULL_unscaled': 1},
 'ComBat_gene': {'HVG_scaled': 1,
  'HVG_unscaled': 3,
  'FULL_scaled': 0,
  'FULL_unscaled': 3},
 'MNN_gene': {'HVG_scaled': 3,
  'HVG_unscaled': 3,
  'FULL_scaled': 0,
  'FULL_unscaled': 0},
 'Scanorama_gene': {'HVG_scaled': 5,
  'HVG_unscaled': 2,
  'FULL_scaled': 0,
  'FULL_unscaled': 0},
 'Harmony_embed': {'HVG_scaled': 3,
  'HVG_unscaled': 1,
  'FULL_scaled': 1,
  'FULL_unscaled': 2},
 'Conos_graph': {'HVG_scaled': 0,
  'HVG_unscaled': 3,
  'FULL_scaled': 3,
  'FULL_unscaled': 1},
 'Unintegrated_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 0,
  'FULL_scaled': 0,
  'FULL_unscaled': 7},
 'LIGER_embed': {'HVG_scaled': 0,
  'HVG_unscaled': 3,
  'FULL_scaled': 0,
  'FULL_unscaled': 4},
 'Seurat 

# Look at trajectories

In [18]:
data[files[0]].columns

Index(['Unnamed: 0', 'Method', 'Output', 'Features', 'Scaling',
       'Overall Score', 'Batch Correction', 'PCR batch', 'Batch ASW',
       'graph iLISI', 'graph connectivity', 'kBET', 'Bio conservation',
       'NMI cluster/label', 'ARI cluster/label', 'Cell type ASW',
       'isolated label F1', 'isolated label silhouette', 'graph cLISI',
       'CC conservation', 'HVG conservation', 'trajectory conservation',
       'method_id'],
      dtype='object')

In [19]:
for file in data:
    data[file] = data[file].loc[~np.isnan(data[file]['Overall Score']),:]
    
    data[file]['method_id'] = ['_'.join(data[file][['Method', 'Output', 'Features','Scaling']].values[i]) for i in range(data[file].shape[0])]
    
    if 'trajectory conservation' not in data[file].columns:
        continue

    print(file)
    print('top performers:')
    data[file].sort_values(by='trajectory conservation', ascending=False)[['method_id', 'trajectory conservation']].reset_index().loc[:8]
    print('bottom performers:')
    data[file].sort_values(by='trajectory conservation', ascending=True)[['method_id', 'trajectory conservation']].reset_index().loc[:8]
    print('\n')

../../../Paper/Supplementary Files/Results/immune_cell_hum_mou_summary_scores.csv
top performers:


,index,method_id,trajectory conservation
0,3,scVI_embed_FULL_unscaled,0.937461
1,11,ComBat_gene_FULL_scaled,0.922775
2,10,MNN_gene_FULL_scaled,0.915112
3,5,ComBat_gene_FULL_unscaled,0.908189
4,9,ComBat_gene_HVG_scaled,0.899172
5,7,MNN_gene_HVG_unscaled,0.869389
6,31,Harmony_embed_FULL_unscaled,0.849324
7,1,Scanorama_embed_FULL_scaled,0.842511
8,26,BBKNN_graph_FULL_scaled,0.833860


bottom performers:


,index,method_id,trajectory conservation
0,22,Conos_graph_FULL_unscaled,0.340602
1,18,Conos_graph_HVG_unscaled,0.345803
2,32,LIGER_embed_FULL_unscaled,0.365057
3,14,ComBat_gene_HVG_unscaled,0.400513
4,8,Scanorama_gene_HVG_scaled,0.429069
5,25,Scanorama_gene_FULL_unscaled,0.488941
6,15,Scanorama_gene_HVG_unscaled,0.504163
7,13,Scanorama_embed_HVG_unscaled,0.519381
8,0,Scanorama_embed_HVG_scaled,0.545590




../../../Paper/Supplementary Files/Results/immune_cell_hum_summary_scores.csv
top performers:


,index,method_id,trajectory conservation
0,37,Unintegrated_gene_FULL_unscaled,0.882521
1,4,Scanorama_embed_FULL_unscaled,0.879324
2,7,Conos_graph_FULL_unscaled,0.873814
3,21,Scanorama_gene_FULL_unscaled,0.871638
4,30,MNN_gene_FULL_unscaled,0.866430
5,5,Harmony_embed_HVG_unscaled,0.866057
6,19,ComBat_gene_FULL_unscaled,0.865043
7,2,Scanorama_embed_HVG_unscaled,0.864949
8,1,Conos_graph_HVG_unscaled,0.864343


bottom performers:


,index,method_id,trajectory conservation
0,29,Seurat v3_gene_HVG_unscaled,0.157173
1,33,Seurat v3_gene_HVG_scaled,0.391396
2,23,Scanorama_gene_FULL_scaled,0.558142
3,34,Seurat v3_gene_FULL_unscaled,0.581284
4,38,LIGER_embed_HVG_unscaled,0.586115
5,25,Seurat v3_gene_FULL_scaled,0.646450
6,31,Conos_graph_HVG_scaled,0.762621
7,15,Harmony_embed_HVG_scaled,0.822546
8,35,trVAE_embed_HVG_unscaled,0.822885


More complex human, mouse integration requires more batch correction to integrate and thus prefers scaling, while in the simpler case of only human data, unscaled preprocessing performs better. Full generall performs better than HVG.

In general: Combat, MNN, scanorama are good, but Seurat scVI is not. Scanorama gene also does not perform particularly well here. In general full features perform better at trajectory conservation.


# CC conservation

In [20]:
topVer = dict()

for file in data:
    if 'CC conservation' not in data[file].columns:
        continue
    
    data[file] = data[file].loc[~np.isnan(data[file]['Overall Score']),:]
    
    data[file]['method_id'] = ['_'.join(data[file][['Method', 'Output']].values[i]) for i in range(data[file].shape[0])]
    
    for meth in data[file]['method_id'].unique():
        tmpDat = data[file][['Features', 'Scaling', 'CC conservation']].loc[data[file]['method_id'].isin([meth])]

        if tmpDat.shape[0] < 2:
            continue
        
        if meth not in topVer:
            topVer[meth] = {'HVG_scaled':0, 'HVG_unscaled':0, 'FULL_scaled':0, 'FULL_unscaled':0}

        tmpDat['preproc'] = ['_'.join(tmpDat[['Features', 'Scaling']].values[i]) for i in range(tmpDat.shape[0])]
        
        topVal = tmpDat.sort_values(by='CC conservation', ascending=False)['preproc'].values[0]
        topVer[meth][topVal] += 1 

In [21]:
topVer

{'Scanorama_embed': {'HVG_scaled': 0,
  'HVG_unscaled': 0,
  'FULL_scaled': 1,
  'FULL_unscaled': 4},
 'scVI_embed': {'HVG_scaled': 0,
  'HVG_unscaled': 2,
  'FULL_scaled': 0,
  'FULL_unscaled': 3},
 'BBKNN_graph': {'HVG_scaled': 0,
  'HVG_unscaled': 5,
  'FULL_scaled': 0,
  'FULL_unscaled': 0},
 'ComBat_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 1,
  'FULL_scaled': 0,
  'FULL_unscaled': 4},
 'MNN_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 0,
  'FULL_scaled': 1,
  'FULL_unscaled': 3},
 'Scanorama_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 0,
  'FULL_scaled': 1,
  'FULL_unscaled': 4},
 'Harmony_embed': {'HVG_scaled': 1,
  'HVG_unscaled': 1,
  'FULL_scaled': 2,
  'FULL_unscaled': 1},
 'Conos_graph': {'HVG_scaled': 0,
  'HVG_unscaled': 3,
  'FULL_scaled': 1,
  'FULL_unscaled': 1},
 'LIGER_embed': {'HVG_scaled': 0,
  'HVG_unscaled': 0,
  'FULL_scaled': 0,
  'FULL_unscaled': 4},
 'Seurat v3_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 1,
  'FULL_scaled': 2,
  'FULL_unscaled': 0},
 'trVAE_embe

In [22]:
top8Meth = defaultdict(int)
bot8Meth = defaultdict(int)

for file in data:
    data[file] = data[file].loc[~np.isnan(data[file]['Overall Score']),:]
    
    data[file]['method_id'] = ['_'.join(data[file][['Method', 'Output', 'Features','Scaling']].values[i]) for i in range(data[file].shape[0])]
    
    if 'CC conservation' not in data[file].columns:
        continue

    for meth in data[file].sort_values(by='CC conservation', ascending=False)['method_id'].values[:8]:
        top8Meth[meth] += 1

    for meth in data[file].sort_values(by='CC conservation', ascending=True)['method_id'].values[:8]:
        bot8Meth[meth] += 1

In [23]:
top8Meth
bot8Meth

defaultdict(int,
            {'Unintegrated_gene_FULL_unscaled': 5,
             'MNN_gene_FULL_unscaled': 3,
             'ComBat_gene_FULL_unscaled': 4,
             'Scanorama_gene_FULL_unscaled': 4,
             'Scanorama_embed_FULL_unscaled': 4,
             'ComBat_gene_FULL_scaled': 2,
             'ComBat_gene_HVG_scaled': 1,
             'MNN_gene_HVG_unscaled': 1,
             'MNN_gene_HVG_scaled': 2,
             'Harmony_embed_HVG_unscaled': 1,
             'Scanorama_gene_HVG_scaled': 1,
             'Scanorama_embed_FULL_scaled': 2,
             'Scanorama_gene_HVG_unscaled': 2,
             'Scanorama_embed_HVG_unscaled': 1,
             'scVI_embed_FULL_unscaled': 1,
             'LIGER_embed_HVG_unscaled': 1,
             'Scanorama_embed_HVG_scaled': 1,
             'MNN_gene_FULL_scaled': 1,
             'Harmony_embed_FULL_scaled': 1,
             'Scanorama_gene_FULL_scaled': 1,
             'Harmony_embed_FULL_unscaled': 1})

defaultdict(int,
            {'LIGER_embed_HVG_unscaled': 3,
             'LIGER_embed_FULL_unscaled': 3,
             'scVI_embed_HVG_unscaled': 4,
             'scVI_embed_FULL_unscaled': 3,
             'Harmony_embed_HVG_scaled': 2,
             'Harmony_embed_HVG_unscaled': 2,
             'Scanorama_embed_FULL_scaled': 1,
             'Scanorama_gene_FULL_scaled': 1,
             'trVAE_embed_HVG_unscaled': 2,
             'Seurat v3_gene_FULL_unscaled': 2,
             'trVAE_embed_FULL_unscaled': 2,
             'Seurat v3_gene_HVG_scaled': 3,
             'Harmony_embed_FULL_unscaled': 1,
             'ComBat_gene_HVG_scaled': 1,
             'ComBat_gene_HVG_unscaled': 2,
             'Scanorama_gene_HVG_scaled': 2,
             'Scanorama_embed_HVG_scaled': 2,
             'Seurat v3_gene_HVG_unscaled': 1,
             'Scanorama_embed_HVG_unscaled': 1,
             'MNN_gene_HVG_unscaled': 1,
             'Scanorama_gene_HVG_unscaled': 1})

Full, and especially unscaled data typically conserve more CC variance.

Scanorama, Combat, and MNN perform consistently well

LIGER, scVI, Seurat, and Harmony perform poorly (note: except in hum mou task, scvI performs well there!)

# HVG conservation

In [24]:
topVer = dict()

for file in data:
    if 'HVG conservation' not in data[file].columns:
        continue
    
    tmp = data[file].loc[~np.isnan(data[file]['HVG conservation']),:].copy()
    
    tmp['method_id'] = ['_'.join(tmp[['Method', 'Output']].values[i]) for i in range(tmp.shape[0])]
    
    for meth in tmp['method_id'].unique():
        tmpDat = tmp[['Features', 'Scaling', 'HVG conservation']].loc[tmp['method_id'].isin([meth])]

        if tmpDat.shape[0] < 2:
            continue
        
        if meth not in topVer:
            topVer[meth] = {'HVG_scaled':0, 'HVG_unscaled':0, 'FULL_scaled':0, 'FULL_unscaled':0}

        tmpDat['preproc'] = ['_'.join(tmpDat[['Features', 'Scaling']].values[i]) for i in range(tmpDat.shape[0])]
        
        topVal = tmpDat.sort_values(by='HVG conservation', ascending=False)['preproc'].values[0]
        topVer[meth][topVal] += 1 

In [25]:
topVer

{'ComBat_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 7,
  'FULL_scaled': 0,
  'FULL_unscaled': 0},
 'MNN_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 6,
  'FULL_scaled': 0,
  'FULL_unscaled': 0},
 'Scanorama_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 7,
  'FULL_scaled': 0,
  'FULL_unscaled': 0},
 'Seurat v3_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 5,
  'FULL_scaled': 0,
  'FULL_unscaled': 0}}

In [26]:
top5Meth = defaultdict(int)
bot5Meth = defaultdict(int)

for file in data:
    tmp = data[file].loc[~np.isnan(data[file]['HVG conservation']),:].copy()
    
    tmp['method_id'] = ['_'.join(tmp[['Method', 'Output', 'Features','Scaling']].values[i]) for i in range(tmp.shape[0])]
    
    if 'HVG conservation' not in tmp.columns:
        continue

    for meth in tmp.sort_values(by='HVG conservation', ascending=False)['method_id'].values[:2]:
        top5Meth[meth] += 1

    for meth in tmp.sort_values(by='HVG conservation', ascending=True)['method_id'].values[:2]:
        bot5Meth[meth] += 1

In [27]:
top5Meth
bot5Meth

defaultdict(int,
            {'Unintegrated_gene_FULL_unscaled': 6,
             'Scanorama_gene_HVG_unscaled': 3,
             'Seurat v3_gene_HVG_unscaled': 3,
             'ComBat_gene_HVG_unscaled': 2})

defaultdict(int,
            {'ComBat_gene_FULL_scaled': 5,
             'MNN_gene_FULL_scaled': 5,
             'ComBat_gene_HVG_scaled': 1,
             'Scanorama_gene_HVG_scaled': 1,
             'Scanorama_gene_FULL_unscaled': 1,
             'Scanorama_gene_FULL_scaled': 1})

Generally performs best on HVG unscaled, worst on full, scaled data



In [30]:
data['../../../Paper/Supplementary Files/Results/pancreas_jointnorm_summary_scores.csv'].sort_values(by='Overall Score', ascending=False)[['Method', 'Output', 'Features','Scaling', 'Bio conservation']]

,Method,Output,Features,Scaling,Bio conservation
0,BBKNN,graph,HVG,unscaled,0.629709
1,Seurat v3,gene,HVG,unscaled,0.578660
2,Conos,graph,FULL,scaled,0.733354
3,Seurat v3,gene,HVG,scaled,0.617193
4,Conos,graph,HVG,scaled,0.736805
5,Seurat v3,gene,FULL,unscaled,0.557171
6,Scanorama,embed,HVG,scaled,0.561862
7,BBKNN,graph,FULL,unscaled,0.548977
8,Seurat v3,gene,FULL,scaled,0.538097
9,Harmony,embed,FULL,scaled,0.507602
